In [1]:
import numpy as np
import pandas as pd
import re
from pathlib import Path
import multi

In [2]:
RAW, DATA = '../raw_data/', '../data/'
COLUMN = ['Article Title', 'Abstract', 'DOI']
FILE = DATA + 'origin_data.csv'

In [3]:
#엑셀 파일 리스트 반환
def excel_list(path):
    folder = Path(path).absolute()
    return [f.name for f in folder.iterdir() if f.is_file()]

#특문 없애기(변화시 생성되어서 지워야함)
def clean_text(text):
    return re.sub(r"[^\w\s.,!?;:\-]", "", text)

#wos에서 엑셀 받은것 병합
def prep_concat(excel_list):

    concat_list = []
    #쓸 열 만 남기기
    def cutted_df(excel_name):
        d = pd.read_excel(excel_name)
        d = d[COLUMN]
        d = d.dropna(subset = COLUMN)
        return d
    
    for e in excel_list:
        concat_list.append(cutted_df(RAW + e))
    #중복제거 및 특문 제거
    result = pd.concat(concat_list, axis=0, ignore_index=True)
    result = result.drop_duplicates(subset=COLUMN)
    result['Article Title'] = result['Article Title'].apply(clean_text)
    result['Abstract'] = result['Abstract'].apply(clean_text)
    
    return result
#경로의 엑셀 모두 하나의 csv로 저장
def make_one_csv(e, file_name):
    m = prep_concat(e)
    m.to_csv(file_name, index=False)

In [4]:
#날짜 부여
def make_date_csv(file_name):
    df = pd.read_csv(file_name)
    df = multi.df_multi_process(df, multi.add_date)
    df.to_csv(file_name, index=False)

In [5]:
#각 task용 데이터에 맞게 전처리
def make_lda_data(file_name):
    lda_data = pd.read_csv(file_name)
    lda_data = multi.df_multi_process(lda_data, multi.text_prep, 6)
    lda_data.to_csv(DATA + 'lda_data.csv', index=False)

def make_bertopic_data(file_name):
    bertopic_data = pd.read_csv(file_name)
    bertopic_data.to_csv(DATA + 'bertopic_data.csv', index=False)

In [6]:
#종합 실행
def make_data(raw_path, file_name):
    #make_one_csv(excel_list(raw_path), file_name)
    #make_date_csv(file_name)
    make_lda_data(file_name)
    #make_bertopic_data(file_name)

make_data(RAW, FILE)

d:\miniconda3\envs\tm\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
